In [20]:
import tensorflow as tf
import numpy as np
import pandas as pd 

In [16]:
#helper functions

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1], padding='SAME')

def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W) + b)

def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b


In [23]:
  

df1= pd.read_csv('/home/gsociety/Desktop/The Model/STORAGE/ProcessedDATA1.csv')
f=df1.drop_duplicates()
f.head()
model_cols=['HOMECODE', 'AWAYCODE', '1',
       '2', 'X', 'HT1', 'HT2', 'HTX', 'FS (0:0)', 'FS(0:0)2', 'FS(0:0)X',
       'G(0.5)O', 'G(0.5)U', 'G(1.5)O', 'G(1.5)U', 'G(2.5)O', 'G(2.5)U',
       'G(3.5)O', 'G(3.5)U', 'G(4.5)O', 'G(4.5)U','FT_Results_1_0_2']

mod_data=f.reindex(columns=model_cols).fillna(0)
####################################
t=f.index[-1]+1
t

dataR=mod_data.values
#mod_data.T

In [25]:
def vfl_data(t,p):
    n=int(((100-p)/100)*t)
    print('Train Data:',t-(t-n),'Test Data:',t-n)
    dataTr=dataR[:n]
    dataTe=dataR[n:]
    
    x_train=dataTr[:,0:21]
    x2_train=x_train.reshape(n,7,3)
    #x_train = x2_train[:, np.newaxis, :,:]
    y_train= dataTr[:,21]


    x1_test=dataTe[:,0:21]
    #x2_test=x1_test.reshape(t-n,7,3)
   # x_test = x2_test[:, np.newaxis, :,:]
    x_test = x1_test
    y_test= dataTe[:,21]
    
    return (x_train, y_train), (x_test, y_test)
(x_train, y_train), (x_test, y_test)=vfl_data(t,10)

Train Data: 3050 Test Data: 339


In [28]:
x =x_train


array([[ 10.  ,  12.  ,   1.85, ...,   0.  ,   0.  ,   0.  ],
       [  7.  ,   6.  ,   2.15, ...,   1.18,   0.  ,   0.  ],
       [  8.  ,   2.  ,   3.55, ...,   1.35,   0.  ,   0.  ],
       ..., 
       [  2.  ,   4.  ,   2.05, ...,   1.4 ,   0.  ,   0.  ],
       [  8.  ,  13.  ,   1.6 , ...,   1.25,   0.  ,   0.  ],
       [ 10.  ,   9.  ,   2.4 , ...,   1.14,   0.  ,   0.  ]])

In [35]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, NB_CLASSES)
y_test = np_utils.to_categorical(y_test, NB_CLASSES)

Using TensorFlow backend.


NameError: name 'NB_CLASSES' is not defined

In [32]:
x = tf.placeholder(tf.float32, shape=[None, 21])
y_ = tf.placeholder(tf.float32, shape=[None, 3])

#input

x_image = tf.reshape(x, [-1, 7, 3, 1])

#model # CONV-MAXPOLL-CONV-MAXPOLL /FLATETEN-DENSE- DROPOUT-CLASSES

#conv-Maxpoll * 2
conv1 = conv_layer(x_image, shape=[5, 5, 1, 32])
conv1_pool = max_pool_2x2(conv1)
conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
conv2_pool = max_pool_2x2(conv2)


#Flatten
conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])

#Dense
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

# dropout
keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

#classify output
y_conv = full_layer(full1_drop, 3)

In [33]:
#mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)
#loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy(y_conv,y_))

#Train with Optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

AttributeError: module 'tensorflow.python.ops.nn' has no attribute 'softmax_cross_entropy'

In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(STEPS):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: batch[0],y_: batch[1],keep_prob: 1.0})

            print "step {}, training accuracy {}".format(i, train_accuracy)

        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1],keep_prob: 0.5})

    X = mnist.test.images.reshape(10, 1000, 784)
    Y = mnist.test.labels.reshape(10, 1000, 10)
    
    test_accuracy = np.mean([sess.run(accuracy,feed_dict={x:X[i], y_:Y[i],keep_prob:1.0})
                             for i in range(10)])
print ("test accuracy: {1}".format(test_accuracy))

SyntaxError: invalid syntax (<ipython-input-19-680ba7d8aa6e>, line 8)